In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from IPython.core.display import HTML
HTML('<link href="https://fonts.googleapis.com/css?family=Cabin|Quicksand" rel="stylesheet"><style>.container{width:90% !important; font-family: "Cabin", sans-serif;}em{color: red !important;}</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;} table, td, th {width: 75%; font-size: 100%;}</style>')

# Multi Index or Hierarchical Index
- the MultiIndex object is the hierarchical analogue of the standard Index object
- it is an array of tuples where each tuple is unique
- operations such as `group by` and `pivoting` and `reshaping` can create a multi index

A multi index can be created starting from a list of arrays or tuples via the function `from_tuples`

In [2]:
level1 = ['home','home','home','school','school','school','work','work','work']
level2 = ['morning', 'day', 'evening', 'morning', 'day', 'evening','morning', 'day', 'evening']
multi_index_lists = [level1,level2]

In [3]:
tuples = list(zip(*multi_index_lists))
tuples

[('home', 'morning'),
 ('home', 'day'),
 ('home', 'evening'),
 ('school', 'morning'),
 ('school', 'day'),
 ('school', 'evening'),
 ('work', 'morning'),
 ('work', 'day'),
 ('work', 'evening')]

In [4]:
mindex = pd.MultiIndex.from_tuples(tuples, names=['where', 'when'])
mindex

MultiIndex([(  'home', 'morning'),
            (  'home',     'day'),
            (  'home', 'evening'),
            ('school', 'morning'),
            ('school',     'day'),
            ('school', 'evening'),
            (  'work', 'morning'),
            (  'work',     'day'),
            (  'work', 'evening')],
           names=['where', 'when'])

In [5]:
df = pd.DataFrame(np.random.randint(10, size=9), index=mindex, columns=['values'])
df

values
where  when           
home   morning       6
       day           8
       evening       5
school morning       2
       day           9
       evening       5
work   morning       1
       day           4
       evening       0

- one can pass a list of arrays and build directly a multi index

In [6]:
df = pd.DataFrame(np.random.randn(9), index=multi_index_lists, columns=['values'])
df.index.names = ['where','when']
df

values
where  when             
home   morning  0.137052
       day      1.880287
       evening  0.938707
school morning  0.433112
       day     -0.814519
       evening  0.867986
work   morning  0.901773
       day     -0.667005
       evening -0.211385

- to build every pair of the elements in two iterables one can use the `MultiIndex.from_product()` method

In [7]:
iterables = [['home','computer'],['morning','day', 'evening']]
multi_idx = pd.MultiIndex.from_product(iterables, names=['where', 'when'])
data = np.random.randint(10,size=6)
df = pd.DataFrame(data, index=multi_idx, columns=['values'])
df

values
where    when           
home     morning       9
         day           6
         evening       4
computer morning       2
         day           9
         evening       0

In [8]:
pd.DataFrame(columns=multi_idx)

Empty DataFrame
Columns: [(home, morning), (home, day), (home, evening), (computer, morning), (computer, day), (computer, evening)]
Index: []

# Multi index for columns
- the `columns` argument can receive a multi index  

In [9]:
df = pd.DataFrame(np.random.randint(10, size=(3, 6)), 
                  index=['John', 'Ann', 'Bob'], 
                  columns=multi_idx)
df

where    home             computer            
when  morning day evening  morning day evening
John        0   0       1        7   5       8
Ann         0   2       2        1   4       4
Bob         4   3       6        1   2       0

In [11]:
pd.DataFrame(index=['John', 'Ann', 'Bob'], 
                  columns=multi_idx)

where    home              computer             
when  morning  day evening  morning  day evening
John      NaN  NaN     NaN      NaN  NaN     NaN
Ann       NaN  NaN     NaN      NaN  NaN     NaN
Bob       NaN  NaN     NaN      NaN  NaN     NaN

In [14]:
pd.DataFrame(index=multi_idx, columns=['ann','bob'])

ann  bob
where    when             
home     morning  NaN  NaN
         day      NaN  NaN
         evening  NaN  NaN
computer morning  NaN  NaN
         day      NaN  NaN
         evening  NaN  NaN

In [15]:
iterables = [['smokers','non_smokers', 'quit_smoke'],['morning','day','evening']]
ridx = pd.MultiIndex.from_product(iterables, names=['cohort', 'period_of_day'])
iterables = [['UK','non_UK'],['<18','18-50','>50']]
cidx = pd.MultiIndex.from_product(iterables, names=['nation', 'age_group'])
df = pd.DataFrame(np.arange(9*6).reshape(9, 6), index=ridx, columns=cidx)
df

nation                     UK           non_UK          
age_group                 <18 18-50 >50    <18 18-50 >50
cohort      period_of_day                               
smokers     morning         0     1   2      3     4   5
            day             6     7   8      9    10  11
            evening        12    13  14     15    16  17
non_smokers morning        18    19  20     21    22  23
            day            24    25  26     27    28  29
            evening        30    31  32     33    34  35
quit_smoke  morning        36    37  38     39    40  41
            day            42    43  44     45    46  47
            evening        48    49  50     51    52  53

# Indexing columns with a Multi Index
- you can select data columns using a *partial* label, i.e. identifying only a subgroup
- the selection *drops* levels of the hierarchical index
- this is like selecting a column in a regular DataFrame (obtaining a Series)

In [16]:
df['UK']

age_group                  <18  18-50  >50
cohort      period_of_day                 
smokers     morning          0      1    2
            day              6      7    8
            evening         12     13   14
non_smokers morning         18     19   20
            day             24     25   26
            evening         30     31   32
quit_smoke  morning         36     37   38
            day             42     43   44
            evening         48     49   50

In [18]:
df['UK']['>50']

cohort       period_of_day
smokers      morning           2
             day               8
             evening          14
non_smokers  morning          20
             day              26
             evening          32
quit_smoke   morning          38
             day              44
             evening          50
Name: >50, dtype: int32

# Indexing rows with a Multi Index
- you can select data rows using a *partial* label, i.e. identifying only a subgroup
- to do so you use tuples with `.loc[]`

In [19]:
df.loc['smokers']

nation         UK           non_UK          
age_group     <18 18-50 >50    <18 18-50 >50
period_of_day                               
morning         0     1   2      3     4   5
day             6     7   8      9    10  11
evening        12    13  14     15    16  17

In [23]:
df.loc[('smokers','evening')]

nation  age_group
UK      <18          12
        18-50        13
        >50          14
non_UK  <18          15
        18-50        16
        >50          17
Name: (smokers, evening), dtype: int32

In [77]:
df.loc[('smokers','morning'),'UK']

age_group
<18      0
18-50    1
>50      2
Name: (smokers, morning), dtype: int64

# Slicing with a Multi Index

- slicing is available for multi indices 
- but indices need to be sorted!

In [24]:
df.loc[('non_smokers','morning'):('quit_smoke','evening'),:]

UnsortedIndexError: 'Key length (2) was greater than MultiIndex lexsort depth (0)'

In [25]:
df = df.sort_index()
df

nation                     UK           non_UK          
age_group                 <18 18-50 >50    <18 18-50 >50
cohort      period_of_day                               
non_smokers day            24    25  26     27    28  29
            evening        30    31  32     33    34  35
            morning        18    19  20     21    22  23
quit_smoke  day            42    43  44     45    46  47
            evening        48    49  50     51    52  53
            morning        36    37  38     39    40  41
smokers     day             6     7   8      9    10  11
            evening        12    13  14     15    16  17
            morning         0     1   2      3     4   5

In [26]:
df.loc[('non_smokers','morning'):('quit_smoke','evening'),:]

nation                     UK           non_UK          
age_group                 <18 18-50 >50    <18 18-50 >50
cohort      period_of_day                               
non_smokers morning        18    19  20     21    22  23
quit_smoke  day            42    43  44     45    46  47
            evening        48    49  50     51    52  53

In [28]:
# column indices need to be sorted too!
df.loc[('non_smokers','morning'):('quit_smoke','evening'),('UK','<18'):('non_UK','18-50')]

UnsortedIndexError: 'Key length (2) was greater than MultiIndex lexsort depth (1)'

In [29]:
df = df.sort_index(axis=1)
df

nation                       UK         non_UK        
age_group                 18-50 <18 >50  18-50 <18 >50
cohort      period_of_day                             
non_smokers day              25  24  26     28  27  29
            evening          31  30  32     34  33  35
            morning          19  18  20     22  21  23
quit_smoke  day              43  42  44     46  45  47
            evening          49  48  50     52  51  53
            morning          37  36  38     40  39  41
smokers     day               7   6   8     10   9  11
            evening          13  12  14     16  15  17
            morning           1   0   2      4   3   5

In [30]:
df.loc[('non_smokers','morning'):('quit_smoke','evening'),('UK','<18'):('non_UK','18-50')]

nation                     UK     non_UK
age_group                 <18 >50  18-50
cohort      period_of_day               
non_smokers morning        18  20     22
quit_smoke  day            42  44     46
            evening        48  50     52

# Working with multi index dataframes

- to avoid surprises, sort the indices (both rows and columns) at the very beginning of the elaboration

In [31]:
iterables = [['smokers','non_smokers', 'quit_smoke'],['morning','day','evening']]
ridx = pd.MultiIndex.from_product(iterables, names=['cohort', 'period_of_day'])
iterables = [['UK','non_UK'],['<18','18-50','>50']]
cidx = pd.MultiIndex.from_product(iterables, names=['nation', 'age_group'])
df = pd.DataFrame(np.arange(9*6).reshape(9, 6), index=ridx, columns=cidx)
df

nation                     UK           non_UK          
age_group                 <18 18-50 >50    <18 18-50 >50
cohort      period_of_day                               
smokers     morning         0     1   2      3     4   5
            day             6     7   8      9    10  11
            evening        12    13  14     15    16  17
non_smokers morning        18    19  20     21    22  23
            day            24    25  26     27    28  29
            evening        30    31  32     33    34  35
quit_smoke  morning        36    37  38     39    40  41
            day            42    43  44     45    46  47
            evening        48    49  50     51    52  53

In [32]:
df = df.sort_index().sort_index(axis=1)
df

nation                       UK         non_UK        
age_group                 18-50 <18 >50  18-50 <18 >50
cohort      period_of_day                             
non_smokers day              25  24  26     28  27  29
            evening          31  30  32     34  33  35
            morning          19  18  20     22  21  23
quit_smoke  day              43  42  44     46  45  47
            evening          49  48  50     52  51  53
            morning          37  36  38     40  39  41
smokers     day               7   6   8     10   9  11
            evening          13  12  14     16  15  17
            morning           1   0   2      4   3   5

# Pivoting

- this is useful when data is available in a "record" format, e.g. as a result of logging

In [33]:
where = ['home','school','home','work','work','home','work','school','school']
task = list('133322121')
result = np.random.randint(10, size=len(task))

data = {'where':where, 'task':task, 'result':result}

df = pd.DataFrame(data)
df

,where,task,result
0,home,1,6
1,school,3,7
2,home,3,6
3,work,3,3
4,work,2,3
5,home,2,8
6,work,1,7
7,school,2,5
8,school,1,8


In [34]:
df.pivot(index='task', columns='where', values='result')

where,home,school,work
task,,,
1,6,8,7
2,8,5,3
3,6,7,3


When pivoting

- if the values argument is omitted
- and the DataFrame has more than one column of values
- then we get hierarchical columns 

In [35]:
where = ['home','home','home','school','school','school','work','work','work']
task = list('123123123')
result_term1 = np.random.randint(10, size=len(task))
result_term2 = np.random.randint(10, size=len(task))

data = {'where':where, 'task':task, 'result_term1':result_term1, 'result_term2':result_term2}

df = pd.DataFrame(data)
df

,where,task,result_term1,result_term2
0,home,1,6,6
1,home,2,9,3
2,home,3,8,2
3,school,1,6,8
4,school,2,7,1
5,school,3,9,1
6,work,1,0,8
7,work,2,0,0
8,work,3,7,3


In [36]:
df2 = df.pivot(index='task', columns='where')
df2

result_term1             result_term2            
where         home school work         home school work
task                                                   
1                6      6    0            6      8    8
2                9      7    0            3      1    0
3                8      9    7            2      1    3

In [37]:
df2['result_term1']

where,home,school,work
task,,,
1,6,6,0
2,9,7,0
3,8,9,7


# Groupby

- pandas allows to:
  - split data into groups based on some criteria
  - apply a function to each group independently
  - combine the results

In [38]:
df2 = pd.DataFrame(np.random.randint(4,size=(6,3)), 
                   index=['alice', 'bob', 'chris','david','emily','fred'], 
                   columns=['home','work','school'])
df2

,home,work,school
alice,1,1,1
bob,1,1,0
chris,0,0,2
david,3,2,2
emily,1,2,2
fred,3,0,2


In [40]:
grouped = df2.groupby(['school'])
grouped

In [43]:
grouped = df2.groupby(['school'])
grouped.groups

{0: ['bob'], 1: ['alice'], 2: ['chris', 'david', 'emily', 'fred']}

In [44]:
grouped = df2.groupby(['work','school'])
grouped.groups

{(0, 2): ['chris', 'fred'], (1, 0): ['bob'], (1, 1): ['alice'], (2, 2): ['david', 'emily']}

# Applying a function to a group

- once the groups are done, one can:
  - aggregate: compute a statistic for each group
  <br>e.g.: group sum/mean/size/counts
  - transform: perform some group-specific computations and return a like-indexed object
  <br>e.g.: standardize data (zscore) within a group, or replace NAs within groups with a value derived from each group
  - filter: discard some groups, according to a group-wise computation that evaluates True or False
  <br>e.g.: delete small groups or filter out data based on the group mean

In [45]:
df2 = pd.DataFrame(np.random.randint(4,size=(6,3)), 
                   index=['alice', 'bob', 'chris','david','emily','fred'], 
                   columns=['home','work','school'])
df2

,home,work,school
alice,2,2,1
bob,2,0,0
chris,0,0,3
david,3,1,3
emily,1,2,2
fred,2,3,0


In [46]:
grouped = df2.groupby(['school'])
grouped.count()

,home,work
school,,
0,2,2
1,1,1
2,1,1
3,2,2


In [47]:
df2 = pd.DataFrame(np.random.randint(4,size=(6,3)), 
                   index=['alice', 'bob', 'chris','david','emily','fred'], 
                   columns=['home','work','school'])
df2

,home,work,school
alice,2,1,3
bob,1,1,3
chris,3,2,2
david,1,2,3
emily,1,1,0
fred,0,2,0


In [49]:
grouped = df2.groupby(['school'])
grouped.groups

{0: ['emily', 'fred'], 2: ['chris'], 3: ['alice', 'bob', 'david']}

In [50]:
grouped.count()

,home,work
school,,
0,2,2
2,1,1
3,3,3


In [51]:
grouped.mean()

,home,work
school,,
0,0.500000,1.500000
2,3.000000,2.000000
3,1.333333,1.333333


# GroupBy with MultiIndex

- with hierarchically-indexed data one can group by one of the levels of the hierarchy
- or by multiple levels

In [52]:
iterables = [['smoker','non_smoker'],['morning','midday','afternoon','evening']]
idx = pd.MultiIndex.from_product(iterables, names=['cohort', 'time'])
df = pd.DataFrame((np.random.rand(8, 4)*3).astype(int), index=idx, columns=list('ABCD'))
df

A  B  C  D
cohort     time                 
smoker     morning    1  1  2  1
           midday     0  0  0  1
           afternoon  1  1  2  0
           evening    2  1  2  2
non_smoker morning    2  2  1  2
           midday     2  0  0  0
           afternoon  2  1  0  0
           evening    2  2  0  0

In [53]:
grouped = df.groupby(level=1)
grouped.groups

{'afternoon': [('smoker', 'afternoon'), ('non_smoker', 'afternoon')], 'evening': [('smoker', 'evening'), ('non_smoker', 'evening')], 'midday': [('smoker', 'midday'), ('non_smoker', 'midday')], 'morning': [('smoker', 'morning'), ('non_smoker', 'morning')]}

In [54]:
grouped.mean()

,A,B,C,D
time,,,,
afternoon,1.5,1.0,1.0,0.0
evening,2.0,1.5,1.0,1.0
midday,1.0,0.0,0.0,0.5
morning,1.5,1.5,1.5,1.5


In [55]:
grouped = df.groupby(level='cohort')
grouped.groups

{'non_smoker': [('non_smoker', 'morning'), ('non_smoker', 'midday'), ('non_smoker', 'afternoon'), ('non_smoker', 'evening')], 'smoker': [('smoker', 'morning'), ('smoker', 'midday'), ('smoker', 'afternoon'), ('smoker', 'evening')]}

In [56]:
grouped.mean()

,A,B,C,D
cohort,,,,
non_smoker,2.0,1.25,0.25,0.5
smoker,1.0,0.75,1.50,1.0


In [206]:
grouped = df.groupby(level=[0,1])
grouped.groups

{('non_smoker', 'afternoon'): [('non_smoker', 'afternoon')], ('non_smoker', 'evening'): [('non_smoker', 'evening')], ('non_smoker', 'midday'): [('non_smoker', 'midday')], ('non_smoker', 'morning'): [('non_smoker', 'morning')], ('smoker', 'afternoon'): [('smoker', 'afternoon')], ('smoker', 'evening'): [('smoker', 'evening')], ('smoker', 'midday'): [('smoker', 'midday')], ('smoker', 'morning'): [('smoker', 'morning')]}

In [57]:
grouped.count()

,A,B,C,D
cohort,,,,
non_smoker,4,4,4,4
smoker,4,4,4,4


# Grouping with Index levels and columns

- one can group by a combination of columns and index levels 
- you need to specify the column names as strings and the index levels as a `pd.Grouper` object

In [182]:
df.groupby([pd.Grouper(level=1), 'A']).groups

{('afternoon', 1): [('non_smoker', 'afternoon')], ('afternoon', 2): [('smoker', 'afternoon')], ('evening', 0): [('smoker', 'evening')], ('evening', 1): [('non_smoker', 'evening')], ('midday', 1): [('smoker', 'midday'), ('non_smoker', 'midday')], ('morning', 0): [('smoker', 'morning')], ('morning', 2): [('non_smoker', 'morning')]}

In [183]:
df.groupby([pd.Grouper(level=1), 'A']).count()

B  C  D
time      A         
afternoon 1  1  1  1
          2  1  1  1
evening   0  1  1  1
          1  1  1  1
midday    1  2  2  2
morning   0  1  1  1
          2  1  1  1

# Iterating through groups

- it is possible to get a key,value pair out of the grouped object
- the key is the value by which elements have been grouped
- the value is the reduced data frame: i.e. a subset of the rows that stisfy the grouping condition

In [71]:
df = pd.DataFrame(np.random.randint(4,size=(6,3)), 
                  index=['alice', 'bob', 'chris','david','emily','fred'], 
                  columns=['home','work','school'])
df

,home,work,school
alice,2,1,2
bob,3,0,3
chris,2,0,3
david,1,2,1
emily,3,3,0
fred,1,3,3


In [72]:
grouped = df.groupby(['work'])

for name, group in grouped:
    print('-'*10)
    print(name)
    print(group)

----------
0
       home  work  school
bob       3     0       3
chris     2     0       3
----------
1
       home  work  school
alice     2     1       2
----------
2
       home  work  school
david     1     2       1
----------
3
       home  work  school
emily     3     3       0
fred      1     3       3


# Selecting a group
- select a single group using `get_group()`

In [73]:
grouped.get_group(0)

,home,work,school
bob,3,0,3
chris,2,0,3


# Aggregation
- once the GroupBy object has been created one can invoke a method
  - describe, mean, sum, size, std, max, min, first, last, nth
  <br> Note: these functions will exclude NA values
- or apply a computation on the grouped data using ` aggregate()` or equivalently the `agg()` method
    - with `agg` one can use any function which reduces a Series to a scalar value

In [74]:
df

,home,work,school
alice,2,1,2
bob,3,0,3
chris,2,0,3
david,1,2,1
emily,3,3,0
fred,1,3,3


In [75]:
grouped = df.groupby(['work'])
print(grouped.describe())

      home                                           school                \
     count mean       std  min   25%  50%   75%  max  count mean      std   
work                                                                        
0      2.0  2.5  0.707107  2.0  2.25  2.5  2.75  3.0    2.0  3.0  0.00000   
1      1.0  2.0       NaN  2.0  2.00  2.0  2.00  2.0    1.0  2.0      NaN   
2      1.0  1.0       NaN  1.0  1.00  1.0  1.00  1.0    1.0  1.0      NaN   
3      2.0  2.0  1.414214  1.0  1.50  2.0  2.50  3.0    2.0  1.5  2.12132   

                                 
      min   25%  50%   75%  max  
work                             
0     3.0  3.00  3.0  3.00  3.0  
1     2.0  2.00  2.0  2.00  2.0  
2     1.0  1.00  1.0  1.00  1.0  
3     0.0  0.75  1.5  2.25  3.0  


In [76]:
grouped.aggregate(np.sum)

,home,school
work,,
0,5,6
1,2,2
2,1,1
3,4,3


In [237]:
grouped['school'].aggregate(np.sum)

work
0    2
1    2
2    4
Name: school, dtype: int64

In [238]:
grouped['school'].agg([np.sum, np.mean, np.std])

,sum,mean,std
work,,,
0,2,0.666667,0.57735
1,2,2.000000,NaN
2,4,2.000000,0.00000


In [241]:
grouped.agg([np.mean, np.std])

home              school         
          mean       std      mean      std
work                                       
0     1.666667  1.527525  0.666667  0.57735
1     3.000000       NaN  2.000000      NaN
2     1.500000  0.707107  2.000000  0.00000

In [242]:
def my_func(iterable):
    res = 0
    for it in iterable:
        res += it * 2 - 3
    return res
    
grouped.agg(my_func)

,home,school
work,,
0,1,-5
1,3,1
2,0,2


# Transformation

- The *transform* method returns an object that is indexed in the same way as the original dataframe
- The transform function must 
    - preserve the size of the group chunk or be a scalar (it will then be broadcasted)
    - operate column-by-column on the group chunk

In [221]:
df = pd.DataFrame(np.random.randint(4,size=(6,3)), 
                  index=['alice', 'bob', 'chris','david','emily','fred'], 
                  columns=['home','work','school'])
df

,home,work,school
alice,1,3,3
bob,2,3,3
chris,1,0,2
david,1,2,0
emily,0,1,2
fred,1,1,2


In [228]:
# replace with single scalar computed over the group
df.groupby('school').transform(lambda x: x.mean())

,home,work
alice,1.500000,3.000000
bob,1.500000,3.000000
chris,0.666667,0.666667
david,1.000000,2.000000
emily,0.666667,0.666667
fred,0.666667,0.666667


In [229]:
df.groupby('school').transform(lambda x: (x - x.mean()) / x.std())

,home,work
alice,-0.707107,NaN
bob,0.707107,NaN
chris,0.577350,-1.154701
david,NaN,NaN
emily,-1.154701,0.577350
fred,0.577350,0.577350


# Filtration

- The *filter* method returns a subset of the original data frame
- The filter takes in input a function that returns True or False when applied to the whole group

In [230]:
df = pd.DataFrame(np.random.randint(4,size=(6,3)), 
                  index=['alice', 'bob', 'chris','david','emily','fred'], 
                  columns=['home','work','school'])
df

,home,work,school
alice,3,1,2
bob,2,0,1
chris,0,0,1
david,1,2,2
emily,3,0,0
fred,2,2,2


In [231]:
df.groupby('home').filter(lambda x:len(x)>1)

,home,work,school
alice,3,1,2
bob,2,0,1
emily,3,0,0
fred,2,2,2


- filter allows to preserve the original size of the dataframe
- and fill with NaNs the rows belonging to the groups that do not pass the filter test

In [233]:
df

,home,work,school
alice,3,1,2
bob,2,0,1
chris,0,0,1
david,1,2,2
emily,3,0,0
fred,2,2,2


In [232]:
df.groupby('home').filter(lambda x:len(x)>1, dropna=False)

,home,work,school
alice,3.0,1.0,2.0
bob,2.0,0.0,1.0
chris,NaN,NaN,NaN
david,NaN,NaN,NaN
emily,3.0,0.0,0.0
fred,2.0,2.0,2.0
